In [1]:
! pip3 install pandas pycoingecko 
import pandas as pd
import datetime as dt
import time
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [2]:
tweet_date = "2018-07-18 01:58:00"
coin='dogecoin'
date_format = "%Y-%m-%d %H:%M:%S"
#1 day from current time will produce 5min interval
#1-90 hourly interval
#above 90 days is daily
number_of_days = 45
number_of_hours = 0
tweet_datetime = pd.to_datetime(tweet_date,format=date_format)
tweet_from_timestamp = (tweet_datetime - dt.timedelta(days=number_of_days,hours=number_of_hours)).timestamp()
tweet_to_timestamp = (tweet_datetime + dt.timedelta(days=number_of_days,hours=number_of_hours)).timestamp()

In [3]:
coin_gecko_data = cg.get_coin_market_chart_range_by_id(id=coin,
                                                vs_currency='usd',
                                                from_timestamp=tweet_from_timestamp,
                                                to_timestamp=tweet_to_timestamp)

In [4]:
def get_nth_element():
    return l[1]
parsed_data = {}
timestamps=list(map(lambda x:x[0],coin_gecko_data['prices']))
for key in coin_gecko_data.keys():
    parsed_data[key] = list(map(lambda x:x[1], coin_gecko_data[key]))
parsed_data['timestamps']=timestamps

In [5]:
df=pd.DataFrame(parsed_data)

In [6]:
df['date']=pd.to_datetime(df['timestamps'],origin='unix',unit='ms')

In [7]:
df

,prices,market_caps,total_volumes,timestamps,date
0,0.003536,4.055510e+08,4.630466e+06,1527994004375,2018-06-03 02:46:44.375
1,0.003537,4.056969e+08,4.647641e+06,1527997602273,2018-06-03 03:46:42.273
2,0.003545,4.066036e+08,3.931993e+06,1528001218261,2018-06-03 04:46:58.261
3,0.003531,4.050303e+08,3.953584e+06,1528004802445,2018-06-03 05:46:42.445
4,0.003599,4.127919e+08,4.105703e+06,1528008405184,2018-06-03 06:46:45.184
...,...,...,...,...,...
2242,0.004353,5.045204e+08,9.432780e+07,1535752613499,2018-08-31 21:56:53.499
2243,0.004681,5.441599e+08,1.103172e+08,1535756208582,2018-08-31 22:56:48.582
2244,0.004888,5.665088e+08,1.244128e+08,1535759822095,2018-08-31 23:57:02.095
2245,0.004526,5.248419e+08,1.259893e+08,1535763418165,2018-09-01 00:56:58.165


In [8]:
df.to_csv('./dogecoin_usd_elon.csv',index=False)